# Lab 1 - Overview of embeddings-based retrieval

Welcome! Here's a few notes about the Chroma course notebooks.
 - A number of warnings pop up when running the notebooks. These are normal and can be ignored.
 - Some operations such as calling an LLM or an opeation using generated data return unpredictable results and so your notebook outputs may differ from the video.
  
Enjoy the course!

In [1]:
from helper_utils import word_wrap

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("microsoft_annual_report_2022.pdf")

page = []
for doc in loader.lazy_load():
    page.append(doc)
pdf_texts = [p.page_content.strip() for p in page]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print(word_wrap(pdf_texts[0]))

1 
Dear shareholders, colleagues, customers, and partners:  
We are
living through a period of historic economic, societal, and
geopolitical change. The world in 2022 looks nothing like 
the world in
2019. As I write this, inflation is at a 40-year high, supply chains
are stretched, and the war in Ukraine is 
ongoing. At the same time, we
are entering a technological era with the potential to power awesome
advancements 
across every sector of our economy and society. As the
world’s largest software company, this places us at a historic

intersection of opportunity and responsibility to the world around us.
 
Our mission to empower every person and every organization on the
planet to achieve more has never been more 
urgent or more necessary.
For all the uncertainty in the world, one thing is clear: People and
organizations in every 
industry are increasingly looking to digital
technology to overcome today’s challenges and emerge stronger. And no

company is better positioned to help th

You can view the pdf in your browser [here](./microsoft_annual_report_2022.pdf) if you would like. 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [6]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in Xbox sales and usage. Despite 
these
increases, we remain dedicated to achieving a net-zero future. We
recognize that progress won’t always be linear, 
and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time.  
On the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate 
over 1.3 million cubic meters of volumetric benefits in nine
water basins around the world. Progress toward our zero waste

commitment included diverting more than 15,200 metric tons of solid
waste otherwise headed to landfills and incinerators, 
as well as
launching new Circular Centers to increase reuse and reduce e-waste at
our datacenters.  
We contracted to protect over 17,000 acres of land
(50% more than the land we use to operate), thus achieving our

commitment to protect more land than we use by 2025.

T

In [8]:
from dotenv import load_dotenv


# Load environment variables from a .env file
load_dotenv('D:/Code/AI/.env')

True

In [9]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

increased, due in large part to significant global datacenter
expansions and the growth in xbox sales and usage. despite these
increases, we remain dedicated to achieving a net - zero future. we
recognize that progress won ’ t always be linear, and the rate at which
we can implement emissions reductions is dependent on many factors that
can fluctuate over time. on the path to becoming water positive, we
invested in 21 water replenishment projects that are expected to
generate over 1. 3 million cubic meters of volumetric benefits in nine
water basins around the world. progress toward our zero waste
commitment included diverting more than 15, 200 metric tons of solid
waste otherwise headed to landfills and incinerators, as well as
launching new circular centers to increase reuse and reduce e - waste
at our datacenters. we contracted to protect over 17, 000 acres of land
( 50 % more than the land we use to operate ), thus achieving our
commitment to protect more land than we use by 2025.


In [10]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

[array([ 3.03590577e-02,  3.52204740e-02,  3.92408445e-02, -3.01640034e-02,
        7.97547996e-02, -7.84079507e-02, -1.38946045e-02, -8.48164316e-03,
       -2.07725763e-02,  6.80732578e-02, -5.64192198e-02, -4.59825583e-02,
        5.00935838e-02,  9.19960514e-02, -7.14867115e-02,  3.69151086e-02,
       -1.84578374e-02, -2.67377887e-02, -5.68575934e-02, -2.27062199e-02,
        2.86354925e-02,  2.97920797e-02,  2.42879894e-02,  3.63898370e-03,
       -4.56295311e-02, -2.71636294e-03, -1.74999703e-02,  6.07888214e-03,
       -2.95465235e-02, -1.85396615e-02,  3.44004966e-02,  3.31753045e-02,
        2.48985682e-02, -6.46156296e-02,  4.08608690e-02,  8.27629715e-02,
       -4.36127633e-02, -1.33252814e-02, -2.08016317e-02, -3.75730395e-02,
       -1.02720242e-02, -9.09821987e-02,  5.88043407e-03,  4.22263471e-03,
       -2.44594719e-02, -2.46812105e-02, -3.94150848e-03, -3.84571031e-02,
       -6.99249282e-03, -3.44783589e-02,  1.54419854e-01, -4.97603379e-02,
       -4.70792875e-02, 

In [11]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_2022", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

350

In [12]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

74 note 13 — unearned revenue unearned revenue by segment was as
follows : ( in millions ) june 30, 2022 2021 productivity and business
processes $ 24, 558 $ 22, 120 intelligent cloud 19, 371 17, 710 more
personal computing 4, 479 4, 311 total $ 48, 408 $ 44, 141 changes in
unearned revenue were as follows : ( in millions ) year ended june 30,
2022 balance, beginning of period $ 44, 141 deferral of revenue 110,
455 recognition of unearned revenue ( 106, 188 ) balance, end of period
$ 48, 408 revenue allocated to remaining performance obligations, which
includes unearned revenue and amounts that will be invoiced and
recognized as revenue in future periods, was $ 193 billion as of june
30, 2022, of which $ 189 billion is related to the commercial portion
of revenue. we expect to recognize approximately 45 % of this revenue
over the next 12 months and the remainder thereafter.


that are not sold separately. • we tested the mathematical accuracy of
management ’ s calculations of revenue a

In [ ]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [18]:
from langchain_ollama import ChatOllama
def rag(query, retrieved_documents, model='deepseek-r1:14b'):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    model=ChatOllama(model='deepseek-r1:14b')
    res=model.invoke(messages)
    # response = openai_client.chat.completions.create(
    #     model=model,
    #     messages=messages,
    # )
    # content = response.choices[0].message.content
    return res.content

In [20]:
output = rag(query=query, retrieved_documents=retrieved_documents)
print(query,end='\n \n')
print(word_wrap(output))

What was the total revenue?
 
<think>
Okay, so the user is asking about the total revenue. I remember
from the annual report that there's a section where they break down
revenue by product and service. Let me look through the information
again.

Hmm, in the financial statements part, it says "total revenue"
for each year. For 2022, it's $198,270 million. That should be the
figure the user is looking for. I'll make sure to present that
clearly.
</think>

The total revenue for the year ended June 30, 2022,
was **$198,270 million**.
